# 1. Importing Libraries

In [118]:
import cv2 #opencv
import numpy as np
import os   #helps with path
from matplotlib import pyplot as plt #to use plt.imshow()
import time              #to measure time between frames 
import mediapipe as mp   


# 2 . drawingutil and hands module

In [119]:
mpDraw=mp.solutions.drawing_utils
mpHands=mp.solutions.hands

# 3. Function to detect points on hands and then drawing on the hands

In [120]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB because by default opencv use bgr but we need rgb for mediapipe to process image
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [121]:
def draw_landmarks(image, results):
    if results.multi_hand_landmarks:
        for num, handsLms in enumerate(results.multi_hand_landmarks):
            mpDraw.draw_landmarks(image,handsLms, mpHands.HAND_CONNECTIONS,
                             mpDraw.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mpDraw.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2) )

# 4. function to get handedness ie left or right and to extract point to numpy array

In [122]:
def get_label(index,results):
    label = None
    for idx, classification in enumerate(results.multi_handedness):
        if classification.classification[0].index == index:
            label = classification.classification[0].label
#         print(index,idx,label)
    if label:
        return label
    else:
        if index == 1:
            return get_label(0,results)
        elif index == 0:
            return get_label(1,results)
        else:
            return label
def extract_keypoints(results):
    hands = [np.zeros(21*3),np.zeros(21*3)]
    if results.multi_hand_landmarks:
        for num, handsLms in enumerate(results.multi_hand_landmarks):
            label = get_label(num,results)
            if label == 'Right':
                hands[0] = np.array([[res.x, res.y, res.z] for res in handsLms.landmark]).flatten()
            if label == 'Left':
                hands[1] = np.array([[res.x, res.y, res.z] for res in handsLms.landmark]).flatten()
            
    return np.concatenate(hands)

# 5. Number of videos of dataset and videolength

In [123]:
# Thirty videos worth of data
no_videos = 400

# Videos are going to be 15 frames in length
video_length = 15

# 6. defining path of dataset and also path where we will store the processed data ie folder Edata in current directory

dataset link <a href="https://drive.google.com/drive/folders/1RZaXXy3pr7YLSv1jKFXDgmNC2nzkgV1P?usp=sharing">DATASET<a>

In [80]:
cwd = os.getcwd()
print(cwd)

C:\Users\91963\Desktop\HandSignRecognition\DEMO\newmodel\AJmodel


In [81]:
datapath = os.path.join('../../KgData/asl4g/train/')
os.listdir(datapath)

['A',
 'B',
 'C',
 'D',
 'del',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'nothing',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'space',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z']

In [124]:
#path for numpy array of data
DATA_PATH = os.path.join('AtoZdata')

In [83]:
os.listdir(cwd)

['.ipynb_checkpoints',
 'board1.jpg',
 'HandGestureRecognitionNeuralNetworkA-J.ipynb',
 'HandGestureRecognitionNeuralNetworkAllDataButSingleFrame.ipynb',
 'model31march.h5']

# 7. defining the categories of signs 

In [125]:
categories = np.array(['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','del','space','nothing'])

# 8. making directory to store the processed data in EData folder

In [85]:
for category in categories: 
    for i in range(no_videos):
        try: 
            os.makedirs(os.path.join(DATA_PATH,category, str(i)))
        except:
            pass

In [86]:
# os.walk('Edata')

# 9. checking data is properly fetching  and also the camera feed

In [87]:
for category in categories:
    path = os.path.join(datapath,category)
    for imgpath in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,imgpath))
        img_array = cv2.resize(img_array,(480,640))
        img_array2 = cv2.flip(img_array,1)
        cv2.imshow("frame",img_array)
        if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
cv2.destroyAllWindows()

In [193]:
cap = cv2.VideoCapture(0) #creating video capture object
## Set mediapipe model

while cap.isOpened():
    #reading feed current frame
    ret,frame = cap.read() 
    flipframe = cv2.flip(frame,1)
    cv2.imshow("Open Cv1",flipframe) 
#     cv2.imshow('o2',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'): #wait for 1 milli second to check if q is pressed on keyboard
        break 
cap.release() #release our webcame
cv2.destroyAllWindows()

# 10. Extracting keypoints using extract_keypoint funtion and for every frame   #  for every category we are creating 400 videos and every video contains 15 frames.

In [88]:
with mpHands.Hands() as model:
    for category in categories:
        path = os.path.join(datapath,category)
        adl = os.listdir(path)
        cdl = 0
        for i in range(no_videos):
            b = False
            for j in range(video_length):
                image = cv2.imread(os.path.join(path,adl[cdl]))
#                 image = cv2.flip(image,1)
                image = cv2.resize(image,(640,480))
                image, results = mediapipe_detection(image, model)
                draw_landmarks(image, results)
#                 print(results.multi_handedness)
                cdl += 1
                cv2.putText(image,'f no {} Vno {} c {}'.format(j,i,category), (10,20), 
                                   cv2.FONT_HERSHEY_SIMPLEX,0.5, (150, 0, 255), 1, cv2.LINE_AA)
                cv2.imshow("frame",image)
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, category, str(i), str(j))
                np.save(npy_path, keypoints)
                
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    b = True
                    break
            if b:
                break


    cv2.destroyAllWindows()

# 11. converting categories using one hot encoding and splitting data into train and test 

In [126]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
label_map = {label:num for num, label in enumerate(categories)}

In [127]:
label_map

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9,
 'K': 10,
 'L': 11,
 'M': 12,
 'N': 13,
 'O': 14,
 'P': 15,
 'Q': 16,
 'R': 17,
 'S': 18,
 'T': 19,
 'U': 20,
 'V': 21,
 'W': 22,
 'X': 23,
 'Y': 24,
 'Z': 25,
 'del': 26,
 'space': 27,
 'nothing': 28}

In [128]:
videos, labels = [], []
for category in categories:
    for video in range(no_videos):
        window = []
        for frame_num in range(video_length):
            res = np.load(os.path.join(DATA_PATH, category, str(video), "{}.npy".format(frame_num)))
            window.append(res)
        videos.append(window)
        labels.append(label_map[category])

In [129]:
X = np.array(videos)

In [130]:
X.shape

(11600, 15, 126)

In [131]:
y = to_categorical(labels).astype(int) #one hot encoding to convert categorial variable

In [132]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [133]:
y_test.shape

(2320, 29)

In [134]:
X_train.shape

(9280, 15, 126)

In [105]:
nsamples, nx, ny = X_train.shape
X_train = X_train.reshape((nsamples,nx*ny))



In [110]:
nsamples, nx, ny = X_test.shape
X_test = X_test.reshape((nsamples,nx*ny))

# Random Forest Model

In [98]:
from sklearn.ensemble import RandomForestClassifier


In [112]:
classifier_rf = RandomForestClassifier(random_state=42, n_jobs=-1, max_depth=5,
                                       n_estimators=1000, oob_score=True)

In [113]:
classifier_rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, n_estimators=1000, n_jobs=-1,
                       oob_score=True, random_state=42)

In [114]:
X_train.shape

(9280, 1890)

In [115]:
classifier_rf.oob_score_

0.9660077288941732

In [116]:
classifier_rf.score(X_test,y_test)

0.014655172413793103

In [117]:
rf = RandomForestClassifier(random_state=42, n_jobs=-1)

In [ ]:
params = {
    'max_depth': [2,3,5,10,20,30,40,50,60],
    'min_samples_leaf': [5,10,20,50,100,200,],
    'n_estimators': [10,25,30,50,100,200]
}

# 12. Building model and training

In [135]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Flatten
from tensorflow.keras.callbacks import TensorBoard

In [136]:
import tensorflow as tf
ACCURACY_THRESHOLD = 0.97
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        # print(logs.get('acc'))
        # print(logs.get('categorical_accuracy'))
        if(logs.get('categorical_accuracy') > ACCURACY_THRESHOLD):
            # print(logs.get('acc'))
            # print(logs.get('categorical_accuracy'))
            print("\nReached %2.2f%% accuracy, so stopping training!!" %(ACCURACY_THRESHOLD*100))
            self.model.stop_training = True

# Instantiate a callback object
callbacks = myCallback()

In [137]:
log_dir = os.path.join('trainLogsdNeuralNetworkATOZ26April')
tb_callback = TensorBoard(log_dir=log_dir)

In [138]:
mycallbacks = [callbacks,tb_callback]

In [139]:
model = Sequential()
model.add(Dense(64,activation='relu',input_shape=(15,126)))
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Flatten())
model.add(Dense(categories.shape[0], activation='softmax'))
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [140]:
 model.fit(X_train, y_train, epochs=500, callbacks=mycallbacks)

Epoch 1/500
  1/290 [..............................] - ETA: 0s - loss: 3.3668 - categorical_accuracy: 0.0312WARNING:tensorflow:From C:\Users\91963\anaconda3\envs\pygpu22\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
290/290 [==============================] - 5s 17ms/step - loss: 2.8839 - categorical_accuracy: 0.0905
Epoch 2/500
290/290 [==============================] - 4s 15ms/step - loss: 2.3523 - categorical_accuracy: 0.1901 2s - loss: 2.459
Epoch 3/500
290/290 [==============================] - 5s 16ms/step - loss: 1.9081 - categorical_accuracy: 0.2933 0s - loss: 1.9108 - categorical_accuracy: 0.29
Epoch 4/500
290/290 [==============================] - 4s 15ms/step - loss: 1.5036 - categorical_accuracy: 0.4418 3s - loss: 1.5883 - catego
Epoch 5/500
290/290 [==============================] - 4s 15ms

290/290 [==============================] - 5s 17ms/step - loss: 0.3031 - categorical_accuracy: 0.9231 0s - loss: 0.3061 - categorical_
Epoch 95/500
290/290 [==============================] - 5s 16ms/step - loss: 0.3151 - categorical_accuracy: 0.9224
Epoch 96/500
290/290 [==============================] - 6s 21ms/step - loss: 0.3734 - categorical_accuracy: 0.9083 0s - loss: 0.3813 - catego
Epoch 97/500
290/290 [==============================] - 7s 24ms/step - loss: 0.4263 - categorical_accuracy: 0.8975
Epoch 98/500
290/290 [==============================] - 6s 20ms/step - loss: 0.4091 - categorical_accuracy: 0.8963 0s - loss: 0.4
Epoch 99/500
290/290 [==============================] - 6s 20ms/step - loss: 0.4098 - categorical_accuracy: 0.8996
Epoch 100/500
290/290 [==============================] - 6s 20ms/step - loss: 0.4058 - categorical_accuracy: 0.9032 0s - loss: 0.4210 - categori
Epoch 101/500
290/290 [==============================] - 7s 24ms/step - loss: 0.3010 - categorical_accu

290/290 [==============================] - 6s 22ms/step - loss: 0.2896 - categorical_accuracy: 0.9295
Epoch 219/500
290/290 [==============================] - 6s 22ms/step - loss: 0.2426 - categorical_accuracy: 0.9413
Epoch 220/500
290/290 [==============================] - 6s 22ms/step - loss: 0.3013 - categorical_accuracy: 0.9260
Epoch 221/500
290/290 [==============================] - 7s 23ms/step - loss: 0.3318 - categorical_accuracy: 0.9207
Epoch 222/500
290/290 [==============================] - 6s 20ms/step - loss: 0.2433 - categorical_accuracy: 0.9404
Epoch 223/500
290/290 [==============================] - 6s 21ms/step - loss: 0.3668 - categorical_accuracy: 0.9126
Epoch 224/500
290/290 [==============================] - 6s 19ms/step - loss: 0.2857 - categorical_accuracy: 0.9305
Epoch 225/500
290/290 [==============================] - 6s 20ms/step - loss: 0.3794 - categorical_accuracy: 0.9149
Epoch 226/500
290/290 [==============================] - 6s 21ms/step - loss: 0.2560 -

290/290 [==============================] - 5s 18ms/step - loss: 0.2662 - categorical_accuracy: 0.9331
Epoch 358/500
290/290 [==============================] - 5s 18ms/step - loss: 0.2762 - categorical_accuracy: 0.9343
Epoch 359/500
290/290 [==============================] - 5s 18ms/step - loss: 0.2917 - categorical_accuracy: 0.9303
Epoch 360/500
290/290 [==============================] - 5s 18ms/step - loss: 0.2660 - categorical_accuracy: 0.9370
Epoch 361/500
290/290 [==============================] - 5s 18ms/step - loss: 0.2432 - categorical_accuracy: 0.9413
Epoch 362/500
290/290 [==============================] - 5s 18ms/step - loss: 0.3298 - categorical_accuracy: 0.9205
Epoch 363/500
290/290 [==============================] - 5s 18ms/step - loss: 0.2501 - categorical_accuracy: 0.9403
Epoch 364/500
290/290 [==============================] - 5s 18ms/step - loss: 0.3430 - categorical_accuracy: 0.9254
Epoch 365/500
290/290 [==============================] - 5s 18ms/step - loss: 0.3305 -

290/290 [==============================] - 6s 19ms/step - loss: 0.2307 - categorical_accuracy: 0.9448
Epoch 498/500
290/290 [==============================] - 5s 19ms/step - loss: 0.2269 - categorical_accuracy: 0.9460
Epoch 499/500
290/290 [==============================] - 5s 19ms/step - loss: 0.2261 - categorical_accuracy: 0.9456
Epoch 500/500
290/290 [==============================] - 5s 18ms/step - loss: 0.2727 - categorical_accuracy: 0.9336


In [152]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 15, 64)            8128      
_________________________________________________________________
dense_21 (Dense)             (None, 15, 64)            4160      
_________________________________________________________________
dense_22 (Dense)             (None, 15, 128)           8320      
_________________________________________________________________
dense_23 (Dense)             (None, 15, 256)           33024     
_________________________________________________________________
dense_24 (Dense)             (None, 15, 512)           131584    
_________________________________________________________________
dense_25 (Dense)             (None, 15, 1024)          525312    
_________________________________________________________________
dense_26 (Dense)             (None, 15, 1024)         

# 13. Make prediction 

In [153]:
res = model.predict(X_test)

In [154]:
categories[np.argmax(res[0])]

'nothing'

In [155]:
categories[np.argmax(y_test[0])]

'nothing'

# 14. Save Model

In [156]:
# model.save('model26April.h5')

# Load Model

In [157]:
model.load_weights('model26April.h5')

# 15. Evaluation using Confusion Matrix and Accuracy


In [158]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [159]:
yhat = model.predict(X_test)

In [160]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [161]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[2227,   11],
        [  14,   68]],

       [[2228,    4],
        [   2,   86]],

       [[2250,    3],
        [  10,   57]],

       [[2254,    3],
        [   3,   60]],

       [[2234,    1],
        [  16,   69]],

       [[2246,    2],
        [   4,   68]],

       [[2243,    1],
        [   1,   75]],

       [[2246,    0],
        [   6,   68]],

       [[2234,    4],
        [  12,   70]],

       [[2245,    1],
        [   5,   69]],

       [[2253,    2],
        [   5,   60]],

       [[2247,    1],
        [   0,   72]],

       [[2238,   11],
        [   8,   63]],

       [[2235,    4],
        [  17,   64]],

       [[2230,    6],
        [  13,   71]],

       [[2227,    3],
        [  13,   77]],

       [[2240,    2],
        [   7,   71]],

       [[2240,    2],
        [  11,   67]],

       [[2221,   10],
        [  14,   75]],

       [[2236,    4],
        [   5,   75]],

       [[2232,    8],
        [  21,   59]],

       [[2215,   12],
        [   

In [162]:
accuracy_score(ytrue, yhat)

0.8931034482758621

# 16. Testing in real Time

In [169]:
colors = [(245,117,16), (117,245,16), (16,117,245),(166,107,245),(106,167,245),(106,117,205),(161,107,25),(130,187,245),(106,117,205),(106,100,245),(196,107,245), (117,245,16), (16,117,245),(245,117,16), (117,245,16), (16,117,245),(166,107,245),(106,167,245),(106,117,205),(161,107,25),(130,187,245),(106,117,205),(106,100,245),(196,107,245), (117,245,16), (16,117,245),(245,117,16), (117,245,16), (16,117,245)]
# def prob_viz(res, actions, input_frame, colors):
#     output_frame = input_frame.copy()
#     for num, prob in enumerate(res):
#         if prob > 90:
#             cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
#             cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2, cv2.LINE_AA)
#     return output_frame

In [170]:
def prob_viz(res, actions,input_frame):
    output_frame = input_frame.copy()
    if res[np.argmax(res)] > 0.9:
        cv2.putText(output_frame,actions[np.argmax(res)], (200,300), cv2.FONT_HERSHEY_SIMPLEX,4, (0, 0, 255), 2, cv2.LINE_AA)
    return output_frame

In [171]:
back = cv2.imread('board1.jpg')
back = cv2.resize(back,(600,600))
# cv2.imshow('back',back)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [172]:
categoriesLatest = np.array(['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','d',' ','n'])

In [173]:
# # 1. New detection variables mode1
# import time
# sequence = []
# sentence = []
# threshold = 0.9

# cap = cv2.VideoCapture(0)
# started = False
# # Set mediapipe model 
# with mpHands.Hands() as hand:
#     i = 20
#     j = 26
#     back2 = cv2.imread('board1.jpg')
#     back2 = cv2.resize(back,(600,600))
#     while cap.isOpened():
#         current_time = time.time()
#         board = back
#         # Read feed
#         ret, frame = cap.read()
#         # Make detections
#         image, results = mediapipe_detection(frame,hand)

        
#         # Draw landmarks
#         draw_landmarks(image, results)
#         # 2. Prediction logic
#         keypoints = extract_keypoints(results)

#         sequence.append(keypoints)
# #         print(sequence)
#         sequence = sequence[-15:]
# #         print(sequence)
        
#         if len(sequence) == 15:
#             res = model.predict(np.expand_dims(sequence, axis=0))[0]
#         #3. Viz logic
#             if res[np.argmax(res)] > threshold: 
#                 casl = categoriesLatest[np.argmax(res)]
#                 if len(sentence) <= 0 or ( casl != 'n' and casl != sentence[-1]):
#                     cv2.putText(back2,casl, (i,j), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
#                     if i <= 20 and  started:
#                         j = j + 26
#                     i = (i + 20)%500
#                     if i == 0:
#                         i = 20
#                     started = True
#                 if len(sentence) > 0: 
#                     if categories[np.argmax(res)] != sentence[-1]:
#                         sentence.append(categoriesLatest[np.argmax(res)])
#                 else:
#                     sentence.append(categoriesLatest[np.argmax(res)]) 

#             if len(sentence) > 10: 
#                 sentence = sentence[-20:]

#             # Viz probabilities
#             board = prob_viz(res, categories,board)
            
        
#         # Show to screen
#         cv2.imshow('OpenCV Feed', image)
#         cv2.imshow('viz',board)
#         cv2.imshow('Text Converter',back2)

#         # Break gracefully
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break

#     cap.release()
#     cv2.destroyAllWindows()

In [65]:
# 1. New detection variables mode 2
import time
sequence = []
sentence = []
threshold = 0.9
time_frame = []
cap = cv2.VideoCapture(0)
started = False
# Set mediapipe model 
with mpHands.Hands() as hand:
    i = 20
    j = 26
    back2 = cv2.imread('board1.jpg')
    back2 = cv2.resize(back,(600,600))
    while cap.isOpened():
        
        board = back
        # Read feed
        ret, frame = cap.read()
        # Make detections
        image, results = mediapipe_detection(frame,hand)

        
        # Draw landmarks
        draw_landmarks(image, results)
        # 2. Prediction logic
        keypoints = extract_keypoints(results)

        sequence.append(keypoints)
#         print(sequence)
        sequence = sequence[-15:]
#         print(sequence)
        
        if len(sequence) == 15:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                casl = categoriesLatest[np.argmax(res)]
                if casl != 'n' and (cv2.waitKey(1) & 0xFF == ord('v')) :
                    cv2.putText(back2,casl, (i,j), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                    if i <= 20 and  started:
                        j = j + 26
                    i = (i + 20)%500
                    if i == 0:
                        i = 20
                    started = True

            # Viz probabilities
            board = prob_viz(res, categories,board)
        # Show to screen
        cv2.imshow('OpenCV Feed', image)
        cv2.imshow('viz',board)
        cv2.imshow('Text Converter',back2)

        # Break gracefully
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [1]:
# 1. New detection variables

sequence = []
sentence = []
threshold = 0.90

cap = cv2.VideoCapture(0)
started = False
# Set mediapipe model
pred_count = {'count':0,'symbol':'n'}
prev_symbol = 'n'
with mpHands.Hands() as hand:
    i = 20
    j = 26
    back2 = cv2.imread('board1.jpg')
    back2 = cv2.resize(back,(600,600))
    while cap.isOpened():
        
        board = back
        # Read feed
        ret, frame = cap.read()
        # Make detections
        image, results = mediapipe_detection(frame,hand)

        
        # Draw landmarks
        draw_landmarks(image, results)
        # 2. Prediction logic
        keypoints = extract_keypoints(results)

        sequence.append(keypoints)
#         print(sequence)
        sequence = sequence[-15:]
#         print(sequence)
        
        if len(sequence) == 15:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                casl = categoriesLatest[np.argmax(res)]
                if casl == pred_count['symbol']:
                    pred_count['count'] += 1
                else:
                    pred_count['symbol'] = casl
                    pred_count['count'] = 0
                if ( casl != 'n'  and pred_count['count'] >= 10 ):
                    cv2.putText(back2,casl, (i,j), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                    if i <= 20 and  started:
                        j = j + 26
                    i = (i + 20)%500
                    if i == 0:
                        i = 20
                    started = True
                    pred_count['count'] = 0

            print(pred_count)


            # Viz probabilities
            board = prob_viz(res, categories,board, colors)
            

        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)
        cv2.imshow('viz',board)
        cv2.imshow('Text Converter',back2)

        # Break gracefully
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break


    cap.release()
    cv2.destroyAllWindows()

NameError: name 'cv2' is not defined

In [51]:
cv2.destroyAllWindows()
cap.release()

In [24]:
np.shape(frame)

(480, 640, 3)

In [25]:
pred_count

{'count': 39, 'symbol': 'C'}

In [57]:
key = cv2.waitKey(10) & 0xFF

In [58]:
back = cv2.imread('board1.jpg')
back = cv2.resize(back,(600,600))
while True:
    cv2.imshow('back',back)
    if key == ord('q'):
        break
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [59]:
cv2.destroyAllWindows()